In [1]:
import pandas as pd
import numpy as np

import math

𝐴𝐶𝑆=Σ(𝐶𝐶𝑖+𝑅𝐶𝑖)×𝐶𝑅𝐹(𝑖𝑟,𝑅)+𝑂&𝑀𝑖)

𝐴𝐶𝑆𝑎𝑑𝑗=[(𝐶𝐶𝑝𝑣+𝐶𝐶𝑏𝑎𝑡)×Δ+𝐶𝐶𝐷𝐺+𝑅𝐶𝑏𝑎𝑡+𝑅𝐶𝐷𝐺]×𝐶𝑅𝐹(𝑖𝑟,𝑅)+𝑂&𝑀𝑝𝑣+𝑂&𝑀𝑏𝑎𝑡+𝑂&𝑀𝐷𝐺

## Indicadores financieros

𝐶𝑅𝐹(𝑖𝑟,𝑅)=𝑖𝑟×(1+𝑖𝑟)^𝑅/(1+𝑖𝑟)^𝑅−1

In [2]:
#i_n = 2 # nominal interest rate
#i_f = 1 # expected annual inflation rate respectively

R = 20 # the life time of the project
ir = 0.0808  #  (i_n - i_f) / (1 + i_f) # Tomado de otro estudio

crf = round(ir*((1+ir)**R) /((1+ir)**R - 1),2)  # The capital recovery factor is calculated by

In [3]:
crf

0.1

## Calculos de Inversión

𝐶𝐶𝑝𝑣=𝑐𝑝𝑣×𝑁𝑝𝑣×𝑃𝑝𝑣𝑠𝑡𝑐
𝐶𝐶𝑏𝑎𝑡=𝑐𝑏𝑎𝑡×𝐸𝑏𝑐𝑒𝑙𝑙_𝑛𝑜𝑚
𝐶𝐶𝐷𝐺=𝑐𝐷𝐺×𝑁𝐷𝐺×𝑤𝐷𝐺

In [4]:
cpv_ins = 1 # costo de PV kWh instalado
cbat_ins = 1 # costo Battery de kWh instalado
cdg_ins = 1 # costo diesel de kWh instalado

npv = 200 # Número de paneles fotovoltaicos
ndg = 1 # Número planta diesel
ppv_stc = 1000 # Capacidad nominal paneles
ebcell_nom = 50.4 # Capacidad de la batería
pdg_rate = 10 # Capacidad nominal diesel

ccpv = round(cpv_ins*npv*ppv_stc,4)
ccbat = round(cbat_ins*ebcell_nom,4)
ccdg = round(cdg_ins*ndg*pdg_rate,4)

## Costos de reemplazo

𝑅𝐶𝑏𝑎𝑡=𝛾𝑏𝑎𝑡×𝐶𝐶𝑏𝑎𝑡×𝐾𝑏𝑎𝑡(𝑖𝑟,𝐿𝑝𝑣,𝑦𝑖)
𝑅𝐶𝐷𝐺=𝛾𝐷𝐺×𝐶𝐶𝐷𝐺×𝐾𝐷𝐺(𝑖𝑟,𝐿𝐷𝐺,𝑦𝑖)

𝐾𝑖(𝑖𝑟,𝐿𝑖,𝑦𝑖)=Σ1(1+𝑖𝑟)𝑛×𝐿𝑖

In [5]:
def calcular_ki(R, li, ir):
    """
    Para cálcular single payment present worth
    """
    yi_replacements = math.floor(R/li)
    values_to_sum = []
    for i in range(1,yi_replacements+1):
        x = (1)/((1+ir)**(i*li))
        values_to_sum.append(x)
        
    return sum(values_to_sum)

kbat = round(calcular_ki(R=20, li=10, ir=0.0808),4)  # single payment present worth battery
kdg = round(calcular_ki(R=20, li=10, ir=0.0808),4) # single payment present worth diesel

In [8]:
ybat = 0.7 #are de-rate factors of the initial capital cost invested 
ydg = 0.7 #are de-rate factors of the initial capital cost invested

rc_bat = round(ybat*ccbat*kbat,4)
rc_dg = round(ydg*ccdg*kdg,4)

## Calculos de operación y Mantto

### Operación y mantto del PV y BATTERY


In [ ]:
factor_pv = 0.1 # Factor de la inversión inicial
factor_bat = 0.1 # Factor de la inversión inicial

oym_pv = factor_pv*ccpv 
oym_bat = factor_bat*ccbat

### Operación y mantto del DIESEL